# Statistical Rethinking: Excercise 2M2

Like in the previous notebook for 2M1, we have the following globe tossing results:

(1) W, W, W <br>
(2) W, W, W, L <br>
(3) L, W, W, L, W, W, W <br>

<br>
We want to estimate the probability of Water $p$ as parameter of the model. The parameter is at least 0 and at most 1.
<br>
<br>
The key point for this exercise is that we employ a different prior:
$$    \text{prior}(p) =
    \begin{cases}
      0, & \text{if}\ p < 0.5 \\
      1, & \text{otherwise}
    \end{cases}$$

In [1]:
require "rubythinking"
grid_size = 50
step_size = 1.0 / grid_size.to_f
grid = 0.step(by: step_size, to: 1).to_a

[0.0, 0.02, 0.04, 0.06, 0.08, 0.1, 0.12, 0.14, 0.16, 0.18, 0.2, 0.22, 0.24, 0.26, 0.28, 0.3, 0.32, 0.34, 0.36, 0.38, 0.4, 0.42, 0.44, 0.46, 0.48, 0.5, 0.52, 0.54, 0.56, 0.58, 0.6, 0.62, 0.64, 0.66, 0.68, 0.7000000000000001, 0.72, 0.74, 0.76, 0.78, 0.8, 0.8200000000000001, 0.84, 0.86, 0.88, 0.9, 0.92, 0.9400000000000001, 0.96, 0.98, 1.0]

In [2]:
prior = grid.map do |x| 
  y =
    if x < 0.5
      0
    else
      1.to_f / (grid_size.to_f / 2.0)
    end
  
  [x, y]
end

area_chart(prior)

"          <script name=\"iruby-chartkick-charting-engine-loaders\" type=\"text/javascript\">\n            /*! For license information please see bundle.js.LICENSE.txt */\n(()=>{var t={616:function(t){t.exports=function(){\"use strict\";function t(t){return\"[object Array]\"===Object.prototype.toString.call(t)}function e(t){return t instanceof Function}function i(t){return\"[object Object]\"===Object.prototype.toString.call(t)&&!e(t)&&t instanceof Object}function n(e,r){var o;for(o in r)\"__proto__\"!==o&&(i(r[o])||t(r[o])?(i(r[o])&&!i(e[o])&&(e[o]={}),t(r[o])&&!t(e[o])&&(e[o]=[]),n(e[o],r[o])):void 0!==r[o]&&(e[o]=r[o]))}function r(t,e){var i={};return n(i,t),n(i,e),i}var o=/^(\\d\\d\\d\\d)(-)?(\\d\\d)(-)?(\\d\\d)$/i;function a(t){return\"\"+t}function s(t){return parseFloat(t)}function l(t){var e,i,n,r;if(\"object\"!=typeof t)if(\"number\"==typeof t)t=new Date(1e3*t);else{if(e=(t=a(t)).match(o))return i=parseInt(e[1],10),n=parseInt(e[3],10)-1,r=parseInt(e[5],10),new Date(i,n,r);var s=t.replace(/ /,\"T\").replace(\" \",\"\").replace(\"UTC\",\"Z\");t=new Date(Date.parse(s)||t)}return t}function h(e){if(!t(e)){var i,n=[];for(i in e)e.hasOwnProperty(i)&&n.push([i,e[i]]);e=n}return e}function c(t,e,i,n,o,a,s,l){return function(h,c,u){var d=h.data,f=r({},t);return f=r(f,u||{}),(h.singleSeriesFormat||\"legend\"in c)&&e(f,c.legend,h.singleSeriesFormat),c.title&&i(f,c.title),\"min\"in c?n(f,c.min):function(t){var e,i,n;for(e=0;e<t.length;e++)for(n=t[e].data,i=0;i<n.length;i++)if(n[i][1]<0)return!0;return!1}(d)||n(f,0),c.max&&o(f,c.max),\"stacked\"in c&&a(f,c.stacked),c.colors&&(f.colors=c.colors),c.xtitle&&s(f,c.xtitle),c.ytitle&&l(f,c.ytitle),r(f,c.library||{})}}function u(t,e){return t[0].getTime()-e[0].getTime()}function d(t,e){return t[0]-e[0]}function f(t,e){return t-e}function g(t){return 0===t.getMilliseconds()&&0===t.getSeconds()}function p(t){return g(t)&&0===t.getMinutes()}function m(t){return p(t)&&0===t.getHours()}function b(t,e){return m(t)&&t.getDay()===e}function x(t){return m(t)&&1===t.getDate()}function y(t){return x(t)&&0===t.getMonth()}function v(t){return!isNaN(l(t))&&a(t).length>=6}function w(t){return\"number\"==typeof t}var _=[\"bytes\",\"KB\",\"MB\",\"GB\",\"TB\",\"PB\",\"EB\"];function M(t,e,i,n){t=t||\"\",i.prefix&&(e<0&&(e*=-1,t+=\"-\"),t+=i.prefix);var r=i.suffix||\"\",o=i.precision,s=i.round;if(i.byteScale){var l,h=n?i.byteScale:e;h>=0x1000000000000000?(e/=0x1000000000000000,l=6):h>=0x4000000000000?(e/=0x4000000000000,l=5):h>=1099511627776?(e/=1099511627776,l=4):h>=1073741824?(e/=1073741824,l=3):h>=1048576?(e/=1048576,l=2):h>=1024?(e/=1024,l=1):l=0,void 0===o&&void 0===s&&(e>=1023.5&&l<_.length-1&&(e=1,l+=1),o=e>=1e3?4:3),r=\" \"+_[l]}if(void 0!==o&&void 0!==s)throw Error(\"Use either round or precision, not both\");if(!n&&(void 0!==o&&(e=e.toPrecision(o),i.zeros||(e=parseFloat(e))),void 0!==s))if(s<0){var c=Math.pow(10,-1*s);e=parseInt((1*e/c).toFixed(0))*c}else e=e.toFixed(s),i.zeros||(e=parseFloat(e));if(i.thousands||i.decimal){var u=(e=a(e)).split(\".\");e=u[0],i.thousands&&(e=e.replace(/\\B(?=(\\d{3})+(?!\\d))/g,i.thousands)),u.length>1&&(e+=(i.decimal||\".\")+u[1])}return t+e+r}function k(t,e,i){return i in e?e[i]:i in t.options?t.options[i]:null}var C={maintainAspectRatio:!1,animation:!1,plugins:{legend:{},tooltip:{displayColors:!1,callbacks:{}},title:{font:{size:20},color:\"#333\"}},interaction:{}},T={scales:{y:{ticks:{maxTicksLimit:4},title:{font:{size:16},color:\"#333\"},grid:{}},x:{grid:{drawOnChartArea:!1},title:{font:{size:16},color:\"#333\"},time:{},ticks:{}}}},D=[\"#3366CC\",\"#DC3912\",\"#FF9900\",\"#109618\",\"#990099\",\"#3B3EAC\",\"#0099C6\",\"#DD4477\",\"#66AA00\",\"#B82E2E\",\"#316395\",\"#994499\",\"#22AA99\",\"#AAAA11\",\"#6633CC\",\"#E67300\",\"#8B0707\",\"#329262\",\"#5574A6\",\"#651067\"],S=function(t,e,i){void 0!==e?(t.plugins.legend.display=!!e,e&&!0!==e&&(t.plugins.legend.position=e)):i&&(t.plugins.legend.display=!1)},P=function(t,e){t.plugins.title.display=!0,t.plugins.title.

In [6]:
factorial = ->(n) do
  return 1 if n < 1
  n.to_i.downto(1).inject(:*)
end


likelihood = ->(w, l, p) do
  (factorial[w+l].to_f / (factorial[w] * factorial[l])).to_f * (p**w) * ((1-p)**l)
end

#<Proc:0x000000011ee831f8 (irb):6 (lambda)>

Now, let's compute the grid aprroximation of the posterior for each of the cases. The difference is only the data input we give in terms of "count of Water" versus "count of Land" of our tossing result given in the exercise.

In [9]:
# For case (1)
w = 3
l = 0

print_posterior = ->(w, l) do
  u_posterior = grid.map do |x| 
    prior_x = prior.detect { |p| p.first == x }.last
    y = prior_x * likelihood[w, l, x]
    {x: x, y: y}
  end

  posterior = u_posterior.map do |pair| 
    y = pair[:y].to_f / u_posterior.map { |pair| pair[:y] }.sum.to_f
   [pair[:x], y]
  end.to_h

  area_chart(posterior)
end

print_posterior[3, 0]

"<div id=\"iruby-chartkick-areachart-15659\" style=\"height: 300px; width: 100%; text-align: center; color: #999; line-height: 300px; font-size: 14px; font-family: 'Lucida Grande', 'Lucida Sans Unicode', Verdana, Arial, Helvetica, sans-serif;\">Loading...</div>\n<script>\n  (function() {\n    if (document.documentElement.hasAttribute(\"data-turbolinks-preview\")) return;\n    if (document.documentElement.hasAttribute(\"data-turbo-preview\")) return;\n\n    var createChart = function() { new Chartkick[\"AreaChart\"](\"iruby-chartkick-areachart-15659\", [[0.0,0.0],[0.02,0.0],[0.04,0.0],[0.06,0.0],[0.08,0.0],[0.1,0.0],[0.12,0.0],[0.14,0.0],[0.16,0.0],[0.18,0.0],[0.2,0.0],[0.22,0.0],[0.24,0.0],[0.26,0.0],[0.28,0.0],[0.3,0.0],[0.32,0.0],[0.34,0.0],[0.36,0.0],[0.38,0.0],[0.4,0.0],[0.42,0.0],[0.44,0.0],[0.46,0.0],[0.48,0.0],[0.5,0.010175010175010175],[0.52,0.011445502645502646],[0.54,0.01281758241758242],[0.56,0.0142951566951567],[0.58,0.015882132682132677],[0.6,0.01758241758241758],[0.62,0.0193999185999186],[0.64,0.021338542938542943],[0.66,0.023402197802197804],[0.68,0.025594790394790397],[0.7000000000000001,0.027920227920227927],[0.72,0.03038241758241758],[0.74,0.03298526658526658],[0.76,0.03573268213268214],[0.78,0.03862857142857143],[0.8,0.041676841676841686],[0.8200000000000001,0.04488140008140009],[0.84,0.048246153846153834],[0.86,0.05177501017501017],[0.88,0.05547187627187627],[0.9,0.05934065934065935],[0.92,0.06338526658526658],[0.9400000000000001,0.06760960520960523],[0.96,0.07201758241758241],[0.98,0.0766131054131054],[1.0,0.0814000814000814]], {}); };\n    if (\"Chartkick\" in window) {\n      createChart();\n    } else {\n      window.addEventListener(\"chartkick:load\", createChart, true);\n    }\n  })();\n</script>\n"

In [10]:
# For case (2)
w = 3
l = 1
print_posterior[3, 1]

"<div id=\"iruby-chartkick-areachart-70078\" style=\"height: 300px; width: 100%; text-align: center; color: #999; line-height: 300px; font-size: 14px; font-family: 'Lucida Grande', 'Lucida Sans Unicode', Verdana, Arial, Helvetica, sans-serif;\">Loading...</div>\n<script>\n  (function() {\n    if (document.documentElement.hasAttribute(\"data-turbolinks-preview\")) return;\n    if (document.documentElement.hasAttribute(\"data-turbo-preview\")) return;\n\n    var createChart = function() { new Chartkick[\"AreaChart\"](\"iruby-chartkick-areachart-70078\", [[0.0,0.0],[0.02,0.0],[0.04,0.0],[0.06,0.0],[0.08,0.0],[0.1,0.0],[0.12,0.0],[0.14,0.0],[0.16,0.0],[0.18,0.0],[0.2,0.0],[0.22,0.0],[0.24,0.0],[0.26,0.0],[0.28,0.0],[0.3,0.0],[0.32,0.0],[0.34,0.0],[0.36,0.0],[0.38,0.0],[0.4,0.0],[0.42,0.0],[0.44,0.0],[0.46,0.0],[0.48,0.0],[0.5,0.030333668411168073],[0.52,0.032756401520313755],[0.54,0.03515475121344381],[0.56,0.03750262568233768],[0.58,0.03977206941818762],[0.6,0.041933263211598744],[0.62,0.043954524152588936],[0.64,0.04580230563058892],[0.66,0.047441197334442235],[0.68,0.04883392525240524],[0.7000000000000001,0.04994135167214711],[0.72,0.05072247518074984],[0.74,0.05113443066470823],[0.76,0.05113248930992992],[0.78,0.05067005860173534],[0.8,0.04969868232485778],[0.8200000000000001,0.04816804056344327],[0.84,0.04602594970105078],[0.86,0.04321836242065198],[0.88,0.03968936770463141],[0.9,0.035381190834786434],[0.92,0.030234193392327212],[0.9400000000000001,0.024186873257876735],[0.96,0.01717586461147086],[0.98,0.00913593793255812],[1.0,0.0]], {}); };\n    if (\"Chartkick\" in window) {\n      createChart();\n    } else {\n      window.addEventListener(\"chartkick:load\", createChart, true);\n    }\n  })();\n</script>\n"

In [11]:
# For case (3)
w = 5
l = 2
print_posterior[5, 2]

"<div id=\"iruby-chartkick-areachart-61911\" style=\"height: 300px; width: 100%; text-align: center; color: #999; line-height: 300px; font-size: 14px; font-family: 'Lucida Grande', 'Lucida Sans Unicode', Verdana, Arial, Helvetica, sans-serif;\">Loading...</div>\n<script>\n  (function() {\n    if (document.documentElement.hasAttribute(\"data-turbolinks-preview\")) return;\n    if (document.documentElement.hasAttribute(\"data-turbo-preview\")) return;\n\n    var createChart = function() { new Chartkick[\"AreaChart\"](\"iruby-chartkick-areachart-61911\", [[0.0,0.0],[0.02,0.0],[0.04,0.0],[0.06,0.0],[0.08,0.0],[0.1,0.0],[0.12,0.0],[0.14,0.0],[0.16,0.0],[0.18,0.0],[0.2,0.0],[0.22,0.0],[0.24,0.0],[0.26,0.0],[0.28,0.0],[0.3,0.0],[0.32,0.0],[0.34,0.0],[0.36,0.0],[0.38,0.0],[0.4,0.0],[0.42,0.0],[0.44,0.0],[0.46,0.0],[0.48,0.0],[0.5,0.030230440239998296],[0.52,0.03389640455469191],[0.54,0.03759576306122878],[0.56,0.041257217006775745],[0.58,0.0448015448911555],[0.6,0.04814272579711524],[0.62,0.051189405068867475],[0.64,0.05384672730179831],[0.66,0.05601856160623748],[0.68,0.05761014410818551],[0.7000000000000001,0.058531162649892644],[0.72,0.05869930865318412],[0.74,0.05804432110842732],[0.76,0.05651254765203498],[0.78,0.054072047695399646],[0.8,0.05071826256815431],[0.8200000000000001,0.04648027763865389],[0.84,0.04142770137457235],[0.86,0.03567818630651054],[0.88,0.02940561685750976],[0.9,0.022848989001365234],[0.92,0.016322006712635093],[0.9400000000000001,0.010223420171239313],[0.96,0.005048130684543599],[0.98,0.0013990872898229294],[1.0,0.0]], {}); };\n    if (\"Chartkick\" in window) {\n      createChart();\n    } else {\n      window.addEventListener(\"chartkick:load\", createChart, true);\n    }\n  })();\n</script>\n"